In [21]:
import os
import shutil
from pathlib import Path
from HandsFreeDocking.Wrapper_Docking import PipelineDocking

In [22]:
protein_pdb = Path("./examples/LAG3_Moloc_2.pdb")
ligands_sdf = Path("./examples/Ligands_To_Dock.sdf")
cystal_sdf = Path("./examples/Fake_Crystal.sdf")

docking_dir = Path("./examples/TMP_Docking")

In [23]:
docking = PipelineDocking(
    workdir=docking_dir,
    docking_software=["plants", "rxdock"],
    settings=(10, 4),
    protein_pdb=protein_pdb,
    ligands_input=ligands_sdf,
    crystal_sdf=cystal_sdf,
    protonation_method="cdp"
)

In [24]:
results = docking.run()

2025-05-30 17:51:15,421 - HandsFreeDocking.Wrapper_Docking - INFO - Using SDF input directly: examples/Ligands_To_Dock.sdf
2025-05-30 17:51:15,423 - HandsFreeDocking.Wrapper_Docking - INFO - Starting Plants docking


Preparing protein with Chimera ...


2025-05-30 17:51:16,918 - HandsFreeDocking.Plants_Pipeline - INFO - Preparing ligands using cdp protonation method
2025-05-30 17:51:16,919 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Apigenin from SDF
2025-05-30 17:51:16,992 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing 6-methylflavone from SDF
2025-05-30 17:51:17,012 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Chrysin from SDF
2025-05-30 17:51:17,039 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Luteolin from SDF
2025-05-30 17:51:17,186 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Kaempferol from SDF
2025-05-30 17:51:17,318 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Eriodictyol from SDF
2025-05-30 17:51:17,511 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Fisetin from SDF
2025-05-30 17:51:17,668 - HandsFreeDocking.Plants_Pipeline - INFO - Converting prepared ligands to mol2 format
2025-05-30 17:51:17,68

Preparing protein with Chimera ...


2025-05-30 17:51:22,010 - HandsFreeDocking.RxDock_Pipeline - INFO - Step 3: Defining binding site...
2025-05-30 17:51:22,011 - HandsFreeDocking.RxDock_Pipeline - INFO - Creating RxDock parameter file and defining binding site...
2025-05-30 17:51:22,012 - HandsFreeDocking.RxDock_Pipeline - INFO - Running command: rbcavity -W -d -r examples/TMP_Docking/RxDock/rxdock.prm
2025-05-30 17:51:22,559 - HandsFreeDocking.Wrapper_Docking - ERROR - Error running rxdock docking: 'ascii' codec can't decode byte 0xc4 in position 160: ordinal not in range(128)


In [25]:
df = docking.concat_df()

In [26]:
df

,ID,Score,Molecule,Software,Protein_Path
0,Eriodictyol_Iso0_Plants-P1,0.000000,<rdkit.Chem.rdchem.Mol object at 0x7fb9abfabf40>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
1,Eriodictyol_Iso0_Plants-P2,0.015162,<rdkit.Chem.rdchem.Mol object at 0x7fb9abfaba00>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
2,Eriodictyol_Iso0_Plants-P3,0.300743,<rdkit.Chem.rdchem.Mol object at 0x7fb9abfabed0>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
3,Eriodictyol_Iso0_Plants-P4,0.340907,<rdkit.Chem.rdchem.Mol object at 0x7fb9abfaac00>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
4,Eriodictyol_Iso0_Plants-P5,0.377505,<rdkit.Chem.rdchem.Mol object at 0x7fba794340b0>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
5,Eriodictyol_Iso0_Plants-P6,0.378139,<rdkit.Chem.rdchem.Mol object at 0x7fba79437f40>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
6,Eriodictyol_Iso0_Plants-P7,0.549804,<rdkit.Chem.rdchem.Mol object at 0x7fba79437df0>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
7,Eriodictyol_Iso0_Plants-P8,0.671879,<rdkit.Chem.rdchem.Mol object at 0x7fba79437920>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
8,Eriodictyol_Iso0_Plants-P9,0.977423,<rdkit.Chem.rdchem.Mol object at 0x7fba794375a0>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
9,Eriodictyol_Iso0_Plants-P10,1.000000,<rdkit.Chem.rdchem.Mol object at 0x7fba79436c00>,plants,/home/hitesit/Python_Packages/Docking_Pipeline...
